In [1]:
import os
import re
import torch
import requests
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForMaskedLM

/home/antwon/apps/miniconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/antwon/apps/miniconda3/envs/nlp/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

class Article:
    def __init__(self, article) -> None:
        self.revealed_dict = self.build_revealed()
        self.tokens = self.tokenize(article, keep_space=True)
        self.set_ = set([token.casefold() for token in self.tokens])

    def build_revealed(self):
        revealed_dict = set()
        for word in stopwords:
            revealed_dict.add(word)
        return revealed_dict
    
    def add_revealed(self, word):
        self.revealed_dict.add(word.casefold())
        self.revealed_dict.add(word.capitalize())

    def tokenize(self, article, keep_space=True):
        if keep_space:
            split = re.split('(\W+)', article)
        else:
            split = re.split('\W+',article)
        return split
    
    def sent_tokenize(self, article):
        return re.split('[\.\n]', article)

    def process_article(self, article):
        mask = re.sub('█+','[MASK]',article)
        return mask

    def redact_article(self,bert_mask = False):
        redacted = []
        for t in self.tokens:
            #print(t)
            if re.search('\W+',t):
                redacted.append(t)
            elif t in self.revealed_dict:
                redacted.append(t)
            else:
                if bert_mask:
                    redacted.append('[MASK]')
                else:
                    redacted.append('█' * len(t))
        return redacted

In [3]:
test_input = '''
Architecture (from Latin  architectura; from Ancient Greek  ἀρχιτέκτων (arkhitéktōn) 'architect'; from  ἀρχι- (arkhi-) 'chief', and  τέκτων (téktōn) 'creator') is the art and technique of designing and building, as distinguished from the skills associated with construction. It is both the process and the product of sketching, conceiving, planning, designing, and constructing buildings or other structures. Architectural works, in the material form of buildings, are often perceived as cultural symbols and as works of art. Historical civilizations are often identified with their surviving architectural achievements.The practice, which began in the prehistoric era, has been used as a way of expressing culture for civilizations on all seven continents. For this reason, architecture is considered to be a form of art. Texts on architecture have been written since ancient times. The earliest surviving text on architectural theories is the 1st century AD treatise De architectura by the Roman architect Vitruvius, according to whom a good building embodies firmitas, utilitas, and venustas (durability, utility, and beauty). Centuries later, Leon Battista Alberti developed his ideas further, seeing beauty as an objective quality of buildings to be found in their proportions. Giorgio Vasari wrote Lives of the Most Excellent Painters, Sculptors, and Architects and put forward the idea of style in the Western arts in the 16th century. In the 19th century, Louis Sullivan declared that "form follows function". "Function" began to replace the classical "utility" and was understood to include not only practical but also aesthetic, psychological and cultural dimensions. The idea of sustainable architecture was introduced in the late 20th century.
Architecture began as rural, oral vernacular architecture that developed from trial and error to successful replication. Ancient urban architecture was preoccupied with building religious structures and buildings symbolizing the political power of rulers until Greek and Roman architecture shifted focus to civic virtues. Indian and Chinese architecture influenced forms all over Asia and Buddhist architecture in particular took diverse local flavors. In fact, During the European Middle Ages, pan-European styles of Romanesque and Gothic cathedrals and abbeys emerged while the Renaissance favored Classical forms implemented by architects known by name. Later, the roles of architects and engineers became separated. Modern architecture began after World War I as an avant-garde movement that sought to develop a completely new style appropriate for a new post-war social and economic order focused on meeting the needs of the middle and working classes. Emphasis was put on modern techniques, materials, and simplified geometric forms, paving the way for high-rise superstructures. Many architects became disillusioned with modernism which they perceived as ahistorical and anti-aesthetic, and postmodern and contemporary architecture developed.
Over the years, the field of architectural construction has branched out to include everything from ship design to interior decorating.


== Definitions ==
Architecture can mean:

A general term to describe buildings and other physical structures.
The art and science of designing buildings and (some) nonbuilding structures.
The style of design and method of construction of buildings and other physical structures.
A unifying or coherent form or structure.
Knowledge of art, science, technology, and humanity.
The design activity of the architect, from the macro-level (urban design, landscape architecture) to the micro-level (construction details and furniture). The practice of the architect, where architecture means offering or rendering professional services in connection with the design and construction of buildings, or built environments.


== Theory of architecture ==

The philosophy of architecture is a branch of philosophy of art, dealing with aesthetic value of architecture, its semantics and in relation with development of culture. Many philosophers and theoreticians from Plato to Michel Foucault, Gilles Deleuze, Robert Venturi and Ludwig Wittgenstein have concerned themselves with the nature of architecture and whether or not architecture is distinguished from building.


=== Historic treatises ===
The earliest surviving written work on the subject of architecture is De architectura by the Roman architect Vitruvius in the early 1st century AD. According to Vitruvius, a good building should satisfy the three principles of firmitas, utilitas, venustas, commonly known by the original translation – firmness, commodity and delight. An equivalent in modern English would be:

Durability – a building should stand up robustly and remain in good condition
Utility – it should be suitable for the purposes for which it is used
Beauty – it should be aesthetically pleasingAccording to Vitruvius, the architect should strive to fulfill each of these three attributes as well as possible. Leon Battista Alberti, who elaborates on the ideas of Vitruvius in his treatise, De re aedificatoria, saw beauty primarily as a matter of proportion, although ornament also played a part. For Alberti, the rules of proportion were those that governed the idealized human figure, the Golden mean. The most important aspect of beauty was, therefore, an inherent part of an object, rather than something applied superficially, and was based on universal, recognizable truths. The notion of style in the arts was not developed until the 16th century, with the writing of Giorgio Vasari. By the 18th century, his Lives of the Most Excellent Painters, Sculptors, and Architects had been translated into Italian, French, Spanish, and English.
In the 16th century, Italian Mannerist architect, painter and theorist Sebastiano Serlio wrote Tutte L'Opere D'Architettura et Prospetiva (Complete Works on Architecture and Perspective). This treatise exerted immense influence throughout Europe, being the first handbook that emphasized the practical rather than the theoretical aspects of architecture, and it was the first to catalog the five orders.In the early 19th century, Augustus Welby Northmore Pugin wrote Contrasts (1836) that, as the title suggested, contrasted the modern, industrial world, which he disparaged, with an idealized image of neo-medieval world. Gothic architecture, Pugin believed, was the only "true Christian form of architecture." The 19th-century English art critic, John Ruskin, in his Seven Lamps of Architecture, published 1849, was much narrower in his view of what constituted architecture. Architecture was the "art which so disposes and adorns the edifices raised by men … that the sight of them" contributes "to his mental health, power, and pleasure". For Ruskin, the aesthetic was of overriding significance. His work goes on to state that a building is not truly a work of architecture unless it is in some way "adorned". For Ruskin, a well-constructed, well-proportioned, functional building needed string courses or rustication, at the very least.On the difference between the ideals of architecture and mere construction, the renowned 20th-century architect Le Corbusier wrote: "You employ stone, wood, and concrete, and with these materials you build houses and palaces: that is construction. Ingenuity is at work. But suddenly you touch my heart, you do me good. I am happy and I say: This is beautiful. That is Architecture". Le Corbusier's contemporary Ludwig Mies van der Rohe said "Architecture starts when you carefully put two bricks together. There it begins."


=== Modern concepts ===
The notable 19th-century architect of skyscrapers, Louis Sullivan, promoted an overriding precept to architectural design: "Form follows function". While the notion that structural and aesthetic considerations should be entirely subject to functionality was met with both popularity and skepticism, it had the effect of introducing the concept of "function" in place of Vitruvius' "utility". "Function" came to be seen as encompassing all criteria of the use, perception and enjoyment of a building, not only practical but also aesthetic, psychological and cultural.
Nunzia Rondanini stated, "Through its aesthetic dimension architecture goes beyond the functional aspects that it has in common with other human sciences. Through its own particular way of expressing values, architecture can stimulate and influence social life without presuming that, in and of itself, it will promote social development.... To restrict the meaning of (architectural) formalism to art for art's sake is not only reactionary; it can also be a purposeless quest for perfection or originality which degrades form into a mere instrumentality".Among the philosophies that have influenced modern architects and their approach to building design are Rationalism, Empiricism, Structuralism, Poststructuralism, Deconstruction and Phenomenology.
In the late 20th century a new concept was added to those included in the compass of both structure and function, the consideration of sustainability, hence sustainable architecture. To satisfy the contemporary ethos a building should be constructed in a manner which is environmentally friendly in terms of the production of its materials, its impact upon the natural and built environment of its surrounding area and the demands that it makes upon non-sustainable power sources for heating, cooling, water and waste management, and lighting.



	
'''

In [4]:
class Game:
    def __init__(self):
        self.bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        self.bert_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
        self.article: Article = None
        self.Y = None
        self.Yset = set()
        self.history = {}
    
    def setArticle(self, article):
        self.article = Article(article)
    
    def setY(self, y: str):
        self.Y = y
        split = y.split()
        self.Yset = set()
        for word in split:
            self.Yset.add(word.casefold())
    
    def best_guess_sentence(self, sentence):
        bert_token_input = self.bert_tokenizer(''.join(sentence),return_tensors="pt")
        with torch.no_grad():
            logits = self.bert_model(**bert_token_input).logits
        
        mask_token_index = (bert_token_input.input_ids == self.bert_tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
        predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
        predictions = self.bert_tokenizer.decode(predicted_token_id)

        prediction_tokens = self.article.tokenize(predictions, keep_space=False)
        prediction_tokens = self.remove_stopwords(prediction_tokens)
        prediction_tokens = self.remove_history(prediction_tokens)
        token_counter = Counter(prediction_tokens)
        return token_counter.most_common(1)
    
    def best_guess(self, article):
        guesses = []
        sentences = self.article.sent_tokenize(article)
        for sentence in sentences:
            guesses.append(self.best_guess_sentence(sentence))
        
        sent_guess_dict = {}
        for guess in guesses:
            if len(guess) > 0:
                try:
                    sent_guess_dict[guess[0][0]] += guess[0][1]
                except KeyError:
                    sent_guess_dict[guess[0][0]] = guess[0][1]
        
        sorted_list = sorted(sent_guess_dict.items(), key=lambda x: x[1],reverse=True)
        sorted_list = [x[0] for x in sorted_list]
        return sorted_list
    
    def remove_stopwords(self,tokens):
        token_list = []
        for t in tokens:
            if (not (t in self.article.revealed_dict)) and (len(t) > 1):
                token_list.append(t)
        return token_list

    # Prevent making guesses that have already been guessed (found in history)
    def remove_history(self,tokens):
        token_list = []
        for t in tokens:
            if (not (t in self.history)):
                token_list.append(t)
        return token_list

    def update_history(self,token):
        self.history[token.casefold()] = None
        self.history[token.capitalize()] = None
    
    def evaluateHitRatio():
        pass

n = 10
run = 1
hit_ratio = 0

game = Game()
game.setArticle(test_input)
game.setY('architecture')

PLAY = True
while PLAY:
    masked = ''.join(game.article.redact_article(bert_mask=True))
    guesses =  game.best_guess(masked)
    best_guess = guesses[0]
    guesses = guesses[: n if len(guesses) > n else len(guesses)]
    countHits = 0
    for guess in guesses:
        game.article.add_revealed(guess)
        if guess == game.Y:
            print('Redactle Word is: {}'.format(
                guess
            ))
            PLAY = False
        if guess in game.article.set_:
            countHits += 1
        if not PLAY: break
    
    if not PLAY: break
    print('Run {}: {} hits.'.format(
        run,
        countHits
    ))

    hit_ratio += countHits / n
    run += 1
    if countHits == 0:
        PLAY = False

hit_ratio /= run
print('Hit Ratio: {}, Rank: {}'.format(
    hit_ratio,
    run
))
print( ''.join(game.article.redact_article()))

In [5]:
data_path = '../data/articles/'
article_df = pd.read_csv(data_path + 'dataset.csv')
article_df = article_df[['title','content']]
article_df = article_df[~article_df['title'].str.contains(' ')]
article_df

,title,content
0,Architecture,Architecture (from Latin architectura; from A...
3,Art,"Art is a diverse range of human activity, and ..."
4,Artist,An artist is a person engaged in an activity r...
5,Museum,"A museum ( mew-ZEE-əm; plural museums or, rare..."
9,Literature,"Literature is any collection of written work, ..."
...,...,...
9938,Sailing,"Sailing employs the wind—acting on sails, wing..."
9939,Sail,A sail is a tensile structure—made from fabric...
9941,Shipbuilding,Shipbuilding is the construction of ships and ...
9942,Steamboat,A steamboat is a boat that is propelled primar...


In [6]:
train, test = train_test_split(article_df, test_size=20)
test

,title,content
748,Milk,Milk is a white liquid food produced by the ma...
5121,Kilogram,The kilogram (also kilogramme) is the unit of ...
8112,Bursa,"Bursa (Ancient Greek: Προῦσα, Latin: Prusa, Ot..."
3805,Hemiptera,Hemiptera (; from Ancient Greek hemipterus 'h...
3870,Coelacanth,The coelacanths ( (listen) SEE-lə-kanth) are f...
8351,Kyrgyzstan,"Kyrgyzstan, or the Kyrgyz Republic, is a landl..."
9859,Dye,A dye is a colored substance that chemically b...
3400,Sense,A sense is a biological system used by an orga...
4533,Turnip,The turnip or white turnip (Brassica rapa subs...
5635,Bomb,A bomb is an explosive weapon that uses the ex...


In [7]:
n = 10
mean_hit_ratio = 0
mean_relative_ranking = 0
class EndGame(Exception): pass
for index, row in test.iterrows():
    run = 0
    hit_ratio = 0
    game = Game()
    game.setArticle(row.content)
    game.setY(row.title)
    print('Title: {}'.format(
        game.Y
    ))
    found = False
    try:
        while True:
            masked = ''.join(game.article.redact_article(bert_mask=True))
            guesses =  game.best_guess(masked)
            guesses = guesses[: n if len(guesses) > n else len(guesses)]
            countHits = 0

            for guess in guesses:
                game.article.add_revealed(guess)

                if guess in game.article.set_:
                    countHits += 1
            
            print('Run: {}, Hits: {}, Guesses: {}'.format(
                run+1,
                countHits,
                guesses
            ))
            hit_ratio += countHits / n
            run += 1
            if game.Yset.issubset(game.article.revealed_dict):
                found = True
                raise EndGame
            if countHits == 0 or run == 20:
                raise EndGame
    except EndGame:
        hit_ratio /= run
        if found:
            print('Redactle Title is: {}'.format(
                game.Y
            ))
        else:
            run = 20
            print('Word not found.')
        
        mean_hit_ratio += hit_ratio
        mean_relative_ranking += run
        print('Hit Ratio: {}, Rank: {}'.format(
            hit_ratio,
            run
        ))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Title: Milk
Run: 1, Hits: 8, Guesses: ['women', 'also', 'music', 'children', 'london', '2006', 'population', 'one', 'thes', 'two']
Run: 2, Hits: 9, Guesses: ['used', 'non', 'number', 'english', 'others', 'love', 'people', 'john', '2011', 'country']
Run: 3, Hits: 9, Guesses: ['found', 'food', 'first', 'man', 'smith', '10', '2007', 'source', 'UNK', 'based']
Run: 4, Hits: 6, Guesses: ['common', 'old', 'history', 'however', 'work', 'died', 'use', 'things', 'laws', 'UNK']
Run: 5, Hits: 8, Guesses: ['cases', 'language', 'system', 'UNK', 'mid', 'england', '50', 'song', 'temperature', 'water']
Run: 6, Hits: 7, Guesses: ['UNK', 'uk', 'sun', 'official', 'include', 'according', 'high', 'among', 'better', 'like']
Run: 7, Hits: 7, Guesses: ['UNK', 'world', 'city', 'later', 'although', 'ands', 'new', 'examples', 'could', 'white']
Run: 8, Hits: 6, Guesses: ['UNK', 'australia', 'county', 'leader', 'word', 'york', 'time', 'taken', 'perhaps', '12']
Run: 9, Hits: 7, Guesses: ['000', 'UNK', 'access', '200

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Title: Kilogram
Run: 1, Hits: 6, Guesses: ['world', 'film', 'one', 'two', 'used', 'however', 'god', 'love', 'name', 'word']
Run: 2, Hits: 9, Guesses: ['english', 'man', 'also', 'based', 'part', 'number', 'time', 'state', 'located', 'three']
Run: 3, Hits: 4, Guesses: ['city', 'book', 'universe', 'called', 'point', 'police', 'would', 'space', 'women', 'system']
Run: 4, Hits: 6, Guesses: ['new', 'note', 'use', 'way', 'UNK', 'law', 'location', 'words', 'example', 'reasons']
Run: 5, Hits: 5, Guesses: ['things', 'still', 'UNK', 'parts', 'metric', 'ideas', 'sometimes', 'characters', 'object', 'finally']
Run: 6, Hits: 6, Guesses: ['units', 'UNK', 'terms', 'according', 'theory', 'product', 'cases', 'defined', '45', 'committee']
Run: 7, Hits: 6, Guesses: ['unit', 'army', 'single', 'length', 'derived', 'second', 'term', 'type', 'american', 'associated']
Run: 8, Hits: 5, Guesses: ['first', 'team', 'UNK', 'following', 'per', 'points', 'land', 'definition', 'common', 'case']
Run: 9, Hits: 7, Guesses

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Title: Pesticide
Run: 1, Hits: 8, Guesses: ['also', 'use', 'water', 'women', 'people', 'world', 'music', 'non', 'history', 'children']
Run: 2, Hits: 9, Guesses: ['london', 'based', 'used', 'part', 'good', 'number', 'two', 'person', 'war', 'europe']
Run: 3, Hits: 8, Guesses: ['one', 'persons', 'time', 'others', 'common', 'things', 'name', 'would', 'followed', '1990s']
Run: 4, Hits: 8, Guesses: ['example', 'released', 'uk', 'often', '2008', 'animals', 'word', 'image', 'science', 'include']
Run: 5, Hits: 5, Guesses: ['examples', 'clock', 'food', 'said', 'describe', 'thes', 'defined', 'systems', 'humans', '2006']
Run: 6, Hits: 6, Guesses: ['means', 'transportation', 'system', 'live', 'us', 'weapons', 'first', 'city', 'species', 'objects']
Run: 7, Hits: 6, Guesses: ['fish', 'animal', 'known', 'life', 'similar', 'english', '100', 'point', 'kind', 'location']
Run: 8, Hits: 6, Guesses: ['found', 'case', 'man', 'low', 'plants', 'object', 'royal', 'difficult', 'horse', 'tool']
Run: 9, Hits: 4, G

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Title: Houston
Run: 1, Hits: 9, Guesses: ['music', 'film', 'two', 'women', 'also', 'man', 'one', 'world', 'however', 'new']
Run: 2, Hits: 6, Guesses: ['first', 'city', 'thes', 'album', '2011', 'children', 'based', 'war', 'london', 'song']
Run: 3, Hits: 8, Guesses: ['people', 'population', 'others', '2008', 'band', 'street', 'white', 'chicago', 'love', 'time']
Run: 4, Hits: 10, Guesses: ['said', '10', '100', '2014', 'according', '500', 'member', 'would', 'number', 'populous']
Run: 5, Hits: 6, Guesses: ['000', 'best', 'non', 'district', 'capital', 'home', 'japan', 'name', 'back', 'door']


KeyboardInterrupt: 

In [8]:
mean_hit_ratio /= len(test)
mean_relative_ranking /= len(test)
print('MHR: {}, MRR: {}'.format(
    mean_hit_ratio,
    mean_relative_ranking
))

MHR: 0.5558320802005013, MRR: 16.9
